In [12]:
from typing import List

class A:
    def add(self) -> List[str]:
        return [
            "a",
            "b",
        ]

class B(A):
    def add(self) -> List[str]:
        l = super().add()
        l.extend(["c", "d"])
        return l

b = B()
b.add()

<class 'super'>


['a', 'b', 'c', 'd']

In [24]:
from typing import List

class A:
    def add(self) -> List[str]:
        if hasattr(super(), "add"):
            print("has attr")
        print("no attr")

In [25]:
a = A()
a.add()

no attr


In [71]:
from functools import wraps
from types import FunctionType, ClassMethodDescriptorType

def wrapper(method, bases):
    print("wrapper!")

    @wraps(method)
    @classmethod
    def wrapped(cls, *args, **kwargs):
        print("calling wrapped!!")
        fin = []
        for base in bases:
            print("checking base: ", base)
            if hasattr(base, "add"):
                a = base.add(cls, *args, **kwargs)
                print("a add: ", a)
                fin.extend(a)
            # print("adding base!!!: ", base.add())
        r = method(*args, **kwargs)
        fin.extend(r)
        return fin
    return wrapped

class MetaClass(type):
    def __new__(meta, classname, bases, classDict):
        newClassDict = {}
        print("-----")
        print("class name: ", classname)
        print("class dict: ", classDict)
        print("bases: ", bases)
        # for base in bases:
        #     if hasattr(base, "add"):
        #         print("adding base: ", base)
        #         print("#$# base add: ", base.add())
        for attributeName, attribute in classDict.items():
            print("attributeName: ", attributeName)
            print("attribute: ", attribute)
            print("type attribute: ", type(attribute))
            if isinstance(attribute, classmethod):
                print("!!found a function!!")
                # replace it with a wrapped version
                attribute = wrapper(attribute, bases)
            newClassDict[attributeName] = attribute
        return type.__new__(meta, classname, bases, newClassDict)


class AA():
    def sub(self):
        pass

class A(AA, metaclass=MetaClass):
    @classmethod
    def add(self) -> List[str]:
        return [
            "a",
            "b",
        ]

class B(A):
    @classmethod
    def add(self) -> List[str]:
        return ["c", "d"]

class D(B):
    @classmethod
    def add(self) -> List[str]:
        return ["e", "f"]

b = D()
b.add()


-----
class name:  A
class dict:  {'__module__': '__main__', '__qualname__': 'A', 'add': <classmethod(<function A.add at 0x113b396c0>)>}
bases:  (<class '__main__.AA'>,)
attributeName:  __module__
attribute:  __main__
type attribute:  <class 'str'>
attributeName:  __qualname__
attribute:  A
type attribute:  <class 'str'>
attributeName:  add
attribute:  <classmethod(<function A.add at 0x113b396c0>)>
type attribute:  <class 'classmethod'>
!!found a function!!
wrapper!


AttributeError: readonly attribute

In [34]:
A

__main__.A

In [33]:
A()

In [47]:
class E:
    def __init__(self, l: str, a: str):
        self.a = l
    def p(self):
        print("xz")

class F(E):
    def __init__(self, l: str):
        # super().__init__(l)
        pass

f = F("x")
f.p()

AttributeError: 'F' object has no attribute 'a'

In [75]:
class X:
    def accumulate(method):
        @wraps(method)
        def wrapped(*args, **kwargs):
            r = method(*args, **kwargs)
            print("r: ", r)
            return r
        return wrapped

    def add(self) -> List[str]:
        return ["a", "b"]

In [76]:
class Y(X):
    @accumulate
    def add(self) -> List[str]:
        return ["b", "c"]
    

NameError: name 'accumulate' is not defined

In [120]:
from dataclasses import make_dataclass, is_dataclass, field
from typing import Generic, TypeVar, Type, Dict, Any, Optional
import inspect
from dataclasses_jsonschema import JsonSchemaMixin, T
from simple_parsing.helpers import Serializable
from arc.generic import RuntimeGeneric

M = TypeVar("M")

class DataclassBuilder(RuntimeGeneric, Generic[M]):
    @classmethod
    def build(cls, reg_cls: Type) -> Optional[Type[M]]:
        """Generate a dataclass for a class

        Args:
            cls (Type[&quot;Server&quot;]): A class

        Returns:
            Opts: A dataclass with the init options of the class
        """
        if is_dataclass(cls):
            raise ValueError("cannot create a dataclass from a dataclass")

        sig: inspect.Signature = inspect.signature(reg_cls.__init__)
        fin_params = []
        for param in sig.parameters:
            if param == "self":
                continue
            if sig.parameters[param].default == inspect._empty:
                fin_params.append((param, sig.parameters[param].annotation))
            else:
                fin_params.append(
                    (
                        param,
                        sig.parameters[param].annotation,
                        field(default=sig.parameters[param].default),
                    )  # type: ignore
                )
        if len(fin_params) == 0:
            return None
        generic_args = cls.__args__
        if len(generic_args) == 0:
            return make_dataclass(reg_cls.__name__ + "Opts", fin_params)
        else:
            return make_dataclass(reg_cls.__name__ + "Opts", fin_params, bases=generic_args)

In [127]:
class Foo():

    def __init__(self):
        self.a = a

    @classmethod
    def schema(cls) -> Dict[str, Any]:
        opts= DataclassBuilder[JsonSchemaMixin].build(cls)
        if opts is None:
            return {}
        return opts.json_schema()

    @classmethod
    def opts(cls) -> Optional[Type[Serializable]]:
        return DataclassBuilder[Serializable].build(cls)


In [128]:
Foo.schema()

{}

In [129]:
opts = Foo.opts()

In [132]:
opts

In [134]:
class R:
    def hello(self, a: str) -> str:
        return a + " hello!"

r = R()
r.hello("jj")

'jj hello!'

In [171]:
from functools import wraps
from types import FunctionType

def init_wrapper(method):
    @wraps(method)
    def wrapped(*args, **kwargs):
        selfVar = args[0]
        _kwargs = kwargs.copy()
        names = method.__code__.co_varnames
        _kwargs.update(zip(names[1:], args[1:]))
        if not "_kwargs" in selfVar.__dict__:
            selfVar.__dict__["_kwargs"] = _kwargs
        return method(*args, **kwargs)
    return wrapped

class ServerMetaClass(type):
    def __new__(meta, classname, bases, classDict):
        newClassDict = {}
        for attributeName, attribute in classDict.items():
            if isinstance(attribute, FunctionType):
                if attributeName == "__init__":
                    attribute = init_wrapper(attribute)
            newClassDict[attributeName] = attribute
        return type.__new__(meta, classname, bases, newClassDict)

class A(metaclass=MetaClass):
    def __init__(self, a: str, b: int) -> None:
        self.a = a

    def echo(self, txt: str) -> str:
        return self.a + txt

attribute:  __main__
attr name:  __module__
attribute:  A
attr name:  __qualname__
attribute:  <function A.__init__ at 0x115276b00>
attr name:  __init__
attribute:  <function A.echo at 0x115277ac0>
attr name:  echo


In [172]:
a = A("hello", 5)

In [173]:
a.echo("today")

'hellotoday'

In [174]:
a.__dict__

{'_kwargs': {'a': 'hello', 'b': 5}, 'a': 'hello'}

In [175]:
class B(A):
    def __init__(self, a: str, b: int, c: bool) -> None:
        super().__init__(a, b)

b = B("bird", 1, True)


attribute:  __main__
attr name:  __module__
attribute:  B
attr name:  __qualname__
attribute:  <function B.__init__ at 0x115334700>
attr name:  __init__
attribute:  <cell at 0x115327f40: empty>
attr name:  __classcell__


In [176]:
b._kwargs

{'a': 'bird', 'b': 1, 'c': True}

In [183]:
if isinstance(b, str):
    print("yes")

In [184]:
from pathlib import Path

Path("/config", "config.json")

PosixPath('/config/config.json')

In [204]:
from starlette.applications import Starlette


def wrapper(method):
    print("outer wrap")
    @wraps(method)
    def wrapped(*args, **kwargs):
        print("args: ", args)
        print("kwargs: ", kwargs)
        print("inner wrap")
        return method(*args, **kwargs)
    return wrapped
    
class E:
    @wrapper
    def c(self, a: str) -> None:
        print("yes: ", a)

outer wrap


In [205]:
E.c("help")

args:  ('help',)
kwargs:  {}
inner wrap


TypeError: E.c() missing 1 required positional argument: 'a'

In [206]:
from abc import ABC, abstractmethod, abstractclassmethod

class H(ABC):
    @abstractmethod
    def b(self, c: str) -> bool:
        pass

    def e(self, r: bool) -> str:
        return "yes"



In [213]:
class I(H):
    def b(self, c: str) -> bool:
        return True

In [214]:
import inspect

In [216]:
bases = inspect.getmro(I)

In [218]:
for base in bases:
    print(base)
    print(inspect.isabstract(base))

<class '__main__.I'>
False
<class '__main__.H'>
True
<class 'abc.ABC'>
False
<class 'object'>
False


In [1]:
from arc.serve.server_test import Foo

In [240]:
inspect.getmro(Foo)

(arc.serve.server_test.Foo,
 arc.serve.server.Server,
 abc.ABC,
 arc.serve.server.APIUtil,
 dataclasses_jsonschema.JsonSchemaMixin,
 object)

In [4]:
from arc.serve.server import Server
import inspect

bases = inspect.getmro(Foo)
# bases = bases[1:]
for base in bases:
    print(base)
    print("base str:", str(base))
    if base == Server:
        print("hello!!")
    if isinstance(base, Server):
        print("is instance!")
        print(base.__name__.lower())
    if issubclass(base, Server):
        print("is subs!")

<class 'arc.serve.server_test.Foo'>
base str: <class 'arc.serve.server_test.Foo'>
is subs!
<class 'arc.serve.server.Server'>
base str: <class 'arc.serve.server.Server'>
hello!!
is subs!
<class 'abc.ABC'>
base str: <class 'abc.ABC'>
<class 'arc.serve.server.APIUtil'>
base str: <class 'arc.serve.server.APIUtil'>
<class 'dataclasses_jsonschema.JsonSchemaMixin'>
base str: <class 'dataclasses_jsonschema.JsonSchemaMixin'>
<class 'object'>
base str: <class 'object'>


In [231]:
bases = inspect.getmro(Foo)
bases = bases[1:]
for base in bases:
    if issubclass(base, Server):
        print("base sub: ", base)
        if inspect.isabstract(base):
            print("shorty!: ", base.short_name())

base sub:  <class 'arc.serve.server.Server'>


In [233]:
inspect.isabstract(Foo)

False

In [235]:
from inspect import isabstract


isabstract(Foo)

False

In [5]:
Foo.artifact_type()

yeah


'server'

# HTTP-RPC

In [301]:
from typing import List, Dict, Any, get_type_hints
from functools import wraps
from types import FunctionType
from inspect import signature
from multiprocessing import Process

from starlette.responses import JSONResponse
from starlette.routing import Route

def init_wrapper(method):
    @wraps(method)
    def wrapped(*args, **kwargs):
        self_var = args[0]
        _kwargs = kwargs.copy()
        names = method.__code__.co_varnames
        _kwargs.update(zip(names[1:], args[1:]))
        if "_kwargs" not in self_var.__dict__:
            self_var.__dict__["_kwargs"] = _kwargs
        return method(*args, **kwargs)

    return wrapped
    
def req_wrapper(method):
    @wraps(method)
    def wrapped(*args, **kwargs):
        self_var = args[0]
        _kwargs = kwargs.copy()
        code = method.__code__
        print("code: ", code)
        print("code dict: ", )
        return method(*args, **kwargs)

    return wrapped


def local(method):
    method.local = True
    return method

class ServerMetaClass(type):
    """Server metaclass is the metaclass for the Server type"""

    def __new__(meta, classname, bases, classDict):
        newClassDict = {}
        routes: List[str] = []

        for attributeName, attribute in classDict.items():
            if isinstance(attribute, FunctionType):
                if hasattr(attribute, "local"):
                    print("!!! ignoring: ", attributeName)
                    continue
                
                if attributeName.startswith("__"):
                    continue
                
                if attributeName == "__init__":
                    attribute = init_wrapper(attribute)
                elif not attributeName.startswith("_"):
                    print("attribute: ", attribute)
                    print("sig: ", signature(attribute))
                    print("sig params: ", signature(attribute).parameters)
                    load_lines = []
                    sig = signature(attribute)
                    for k in sig.parameters:
                        print("k", k)
                        if k == "self":
                            continue
                        param = sig.parameters[k]
                        print("param annot: ", param.annotation)
                        _op = getattr(param.annotation, "from_dict", None)
                        print("_op: ", _op)
                        if callable(_op):
                            print("!!!! callable")
                            load_lines.append(f"jdict['{k}'] = {param.annotation.__name__}.from_dict(jdict['{k}'])")

                    hints = get_type_hints(attribute)
                    ret = hints["return"]
                    ret_op = getattr(ret, "to_dict", None)
                    ser_line = ""
                    print("ret_op: ", ret_op)
                    if callable(ret_op):
                        ser_line = "ret = ret.to_dict()"
                        
                    joined_load_lines = "\n".join(load_lines)
                    req_code = f"""
async def _{attributeName}_req(self, request): 
    jdict = await request.json()
    {joined_load_lines}
    ret = {attributeName}(**jdict)
    {ser_line}
    return JSONResponse(ret)
"""
                    print("req code: ", req_code)
                    exec(req_code)
                    newClassDict[f"_{attributeName}_req"] = eval(f"_{attributeName}_req")
                    routes.append(f"Route('/{attributeName}', endpoint=self._{attributeName}_req)")
            newClassDict[attributeName] = attribute
        
        routes_joined = ", ".join(routes)
        route_code = f"""
def _routes(self) -> List[Route]:
    return [{routes_joined}]
"""
        print("route code: ", route_code)
        exec(route_code)
        newClassDict["_routes"] = _routes
        return type.__new__(meta, classname, bases, newClassDict)

In [357]:
import typing
from typing import Type

class Server(metaclass=ServerMetaClass):
    # def _assemble_import(module, name):
    #     return 'import {} as {}'.format(module.__name__, name)
        
    @classmethod
    def gen_client(cls) -> str:
        fns = inspect.getmembers(cls, predicate=inspect.isfunction)
        print("fns: ", fns)
        params: List[str] = []
        methods: List[str] = []
        imports: List[str] = []
        for name, fn in fns:
            if name.startswith("_"):
                continue
            
            print("name: ", name)
            print("fn: ", fn)
            sig = inspect.signature(fn)
            print("sig str: ", str(sig))

            # print("sig: ", sig)
            # print("params: ", sig.parameters)

            for k in sig.parameters:
                if k == "self":
                    continue
                param = sig.parameters[k]
                # print("param annot: ", param.annotation)
                _op = getattr(param.annotation, "to_dict", None)
                if callable(_op):
                    params.append(f"'{k}': {k}.to_dict()")
                    continue

                params.append(f"'{k}': {k}")

            import_statements: Dict[str, Any] = {}
            hints = get_type_hints(fn)

            def proc_hint(h: Type):
                print("===================")
                print("processing hint....")
                print("===================")
                mod = h.__module__
                print("h mod: ", mod)

                if mod == "builtins":
                    return
            

                if isinstance(h, typing._GenericAlias):
                    print("is generic")
                    args = typing.get_args(h)
                    for arg in args:
                        print("processing arg: ", arg)
                        proc_hint(arg)

                    origin = typing.get_origin(h)
                    if origin == list:
                        origin = List
                    if origin == dict:
                        origin = Dict
                    h = origin

                import_statements[f"from {mod} import {h.__name__}"] = None
                
            for hint in hints:
                print("hint: ", hint)
                h = hints[hint]
                proc_hint(h)
            
            ret = hints["return"]
            ret_op = getattr(ret, "from_dict", None)
            ser_line = "ret = jdict['response']"
            print("ret_op: ", ret_op)
            if callable(ret_op):
                ser_line = "ret = ret.from_dict(jdict)"

            params_joined = ",".join(params)

            client_fn = f"""
    def {name}{sig}:
        params = json.dumps({{{params_joined}}}).encode("utf8")
        req = request.Request(f"{{self.server_addr}}/{name}", data=params, headers={{"content-type": "application/json"}})
        resp = request.urlopen(req)
        data = resp.read().decode("utf-8")
        jdict = json.loads(data)
        {ser_line}
        return ret
            """
            methods.append(client_fn)
        
        imports_joined = "\n".join(import_statements.keys())
        methods_joined = "\n".join(methods)
        client_file = f"""
from urllib import request
import json
import logging

from arc.serve.server import Client
{imports_joined}

class {cls.__name__}Client(Client):
{methods_joined}
        """

        return client_file

    @classmethod
    def write_client_file(cls) -> None:
        with open(f"{cls.__name__.lower()}_client.py", 'w', encoding = 'utf-8') as f:
            f.write(cls.gen_client())

route code:  
def _routes(self) -> List[Route]:
    return []



In [358]:
class Model(Server):
    def fit(self, x: List[int], y: List[int], opts: Dict[str, Any]) -> float:
        print("fitting x to y")
        return 1

attribute:  <function Model.fit at 0x12bb8b490>
sig:  (self, x: List[int], y: List[int], opts: Dict[str, Any]) -> float
sig params:  OrderedDict([('self', <Parameter "self">), ('x', <Parameter "x: List[int]">), ('y', <Parameter "y: List[int]">), ('opts', <Parameter "opts: Dict[str, Any]">)])
k self
k x
param annot:  typing.List[int]
_op:  None
k y
param annot:  typing.List[int]
_op:  None
k opts
param annot:  typing.Dict[str, typing.Any]
_op:  None
ret_op:  None
req code:  
async def _fit_req(self, request): 
    jdict = await request.json()
    
    ret = fit(**jdict)
    
    return JSONResponse(ret)

route code:  
def _routes(self) -> List[Route]:
    return [Route('/fit', endpoint=self._fit_req)]



In [359]:
Model.gen_client()
Model.write_client_file()

fns:  [('_fit_req', <function _fit_req at 0x12bb8a050>), ('_routes', <function _routes at 0x12a7ac5e0>), ('fit', <function Model.fit at 0x12bb8b490>)]
name:  fit
fn:  <function Model.fit at 0x12bb8b490>
sig str:  (self, x: List[int], y: List[int], opts: Dict[str, Any]) -> float
hint:  x
processing hint....
h mod:  typing
is generic
processing arg:  <class 'int'>
processing hint....
h mod:  builtins
hint:  y
processing hint....
h mod:  typing
is generic
processing arg:  <class 'int'>
processing hint....
h mod:  builtins
hint:  opts
processing hint....
h mod:  typing
is generic
processing arg:  <class 'str'>
processing hint....
h mod:  builtins
processing arg:  typing.Any
processing hint....
h mod:  typing
hint:  return
processing hint....
h mod:  builtins
ret_op:  None
fns:  [('_fit_req', <function _fit_req at 0x12bb8a050>), ('_routes', <function _routes at 0x12a7ac5e0>), ('fit', <function Model.fit at 0x12bb8b490>)]
name:  fit
fn:  <function Model.fit at 0x12bb8b490>
sig str:  (self, x

In [372]:
from typing import Generic, TypeVar

X = TypeVar('X')

class A(Generic[X]):
    def a(x: X):
        print(x)

b = A[int]

class B:
    def b(a: A[int]):
        print(a)

In [376]:
from typing import get_origin


get_origin(b)
print(type(A[int]))

<class 'typing._GenericAlias'>


In [298]:
h = typing.get_type_hints(B.b)
h

{'a': __main__.A[int]}

In [335]:
typing.get_args(h['a'])

(int,)

In [287]:
type(a)

NoneType

In [239]:
typing._GenericAlias.mro

<function _GenericAlias.mro()>

In [206]:
class B(metaclass=ServerMetaClass):
    pass

route code:  
def _routes(self) -> List[Route]:
    return []



In [220]:
List.__module__

'typing'

In [190]:
class ObjSer:
    def __init__(self, a: str, b: int) -> None:
        self.a = a
        self.b = b

    @classmethod
    def from_dict(cls, d: Dict[str, Any]) -> "ObjSer":
        return cls(**d)

    def to_dict(self) -> Dict[str, Any]:
        return {"a": a, "b": b}

class B(metaclass=ServerMetaClass):
    pass

class A(metaclass=ServerMetaClass):
    """A A class"""

    def __init__(self, a: str, b: int, c: bool, d: List[str], e: Dict[str, Any] = {"int": 1}):
        """Create an A class

        Args:
            a (str): A str
            b (int): B int
            c (bool): C bool
            d (List[str]): D list of str
            e (Dict[str, Any], optional): E dict. Defaults to {"int": 1}.
        """
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        self.e = e

    @local
    def name(self) -> str:
        """Name of the class

        Returns:
            str: The name of the class
        """
        return "A"

    def echo(self, s: str, append: bool = True) -> str:
        """Echo back the str

        Args:
            s (str): Str to echo
            append (bool, optional): Whether to append to the str when echoed

        Returns:
            str: The echoed string
        """
        if append:
            return s + " hello"
        
        return s

    def add(self, x: int, y: int) -> int:
        """Add x and y

        Args:
            x (int): x to add
            y (int): y to add

        Returns:
            int: Sum of the numbers
        """
        return x + y   

    def obj_ser(self, o: ObjSer) -> ObjSer:
        """Test a serializable object

        Args:
            o (ObjSer): A serializable object

        Returns:
            ObjSer: A serializable object
        """
        return o 

    def _bool_to_int(self, c: bool) -> int:
        return int(c) 


route code:  
def _routes(self) -> List[Route]:
    return []

!!! ignoring:  name
attribute:  <function A.echo at 0x12b73fe20>
sig:  (self, s: str, append: bool = True) -> str
sig params:  OrderedDict([('self', <Parameter "self">), ('s', <Parameter "s: str">), ('append', <Parameter "append: bool = True">)])
k self
k s
param annot:  <class 'str'>
_op:  None
k append
param annot:  <class 'bool'>
_op:  None
ret_op:  None
req code:  
async def _echo_req(self, request): 
    jdict = await request.json()
    
    ret = echo(**jdict)
    
    return JSONResponse(ret)

attribute:  <function A.add at 0x12b7dc3a0>
sig:  (self, x: int, y: int) -> int
sig params:  OrderedDict([('self', <Parameter "self">), ('x', <Parameter "x: int">), ('y', <Parameter "y: int">)])
k self
k x
param annot:  <class 'int'>
_op:  None
k y
param annot:  <class 'int'>
_op:  None
ret_op:  None
req code:  
async def _add_req(self, request): 
    jdict = await request.json()
    
    ret = add(**jdict)
    
    return JSON

In [139]:
class B(A):
    def new_meth(self, c: int) -> bool:
        return True

attribute:  <function B.new_meth at 0x12a1ff520>
sig:  (self, c: int) -> bool
sig params:  OrderedDict([('self', <Parameter "self">), ('c', <Parameter "c: int">)])
k self
k c
param annot:  <class 'int'>
_op:  None
ret_op:  None
req code:  
async def _new_meth_req(self, request): 
    jdict = await request.json()
    
    ret = new_meth(**jdict)
    
    return JSONResponse(ret)

route code:  
def _routes(self) -> List[Route]:
    return [Route('/new_meth', endpoint=self.new_meth)]



In [124]:
B.__dict__

mappingproxy({'__module__': '__main__',
              '_new_meth_req': <function __main__._new_meth_req(self, request)>,
              'new_meth': <function __main__.B.new_meth(self, c: int) -> bool>,
              '_routes': <function __main__._routes() -> List[starlette.routing.Route]>,
              '__doc__': None})

In [85]:
j = []
"\n".join(j)

''

In [99]:
t = eval("lambda: print('hello')")

In [92]:
a = c

In [93]:
a()

hello


In [100]:
t()

hello


In [111]:
route_code = f"""
def _routes() -> List[Route]:
    return [Route("/path", endpoint=t),]
"""

In [112]:
exec(route_code)

In [113]:
_routes()

In [178]:
class E:
    def hello(self, a: str) -> bool:
        return True

class F(E):
    def d(self, b: bool) -> int:
        return 1

In [179]:
F.__dict__

mappingproxy({'__module__': '__main__',
              'd': <function __main__.F.d(self, b: bool) -> int>,
              '__doc__': None})

In [184]:
fns = inspect.getmembers(F, predicate=inspect.isfunction)

for name, fn in fns:
    print("name: ", name)
    print("fn: ", fn)

name:  d
fn:  <function F.d at 0x12b73cca0>
name:  hello
fn:  <function E.hello at 0x12b73d750>


In [172]:
dir(F)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'd',
 'hello']

In [127]:
F.hello

<function __main__.E.hello(a: str) -> bool>

In [158]:
class C:
    @staticmethod
    def a() -> List[str]:
        return ["s"]

class D(C):
    @staticmethod
    def a() -> List[str]:
        print(super())

In [159]:
c = C()
c.a()

d = D()
d.a()

RuntimeError: super(): no arguments

In [259]:
C.a()

['s']

In [260]:
from abc import abstractclassmethod

In [261]:
loca = locals().copy()

In [262]:
f = filter(lambda x: inspect.ismodule(x[1]), loca.items())

In [263]:
for e in f:
    print(e)

('__builtin__', <module 'builtins' (built-in)>)
('__builtins__', <module 'builtins' (built-in)>)
('sys', <module 'sys' (built-in)>)
('os', <module 'os' from '/Users/patrickbarker/.pyenv/versions/3.10.1/lib/python3.10/os.py'>)
('inspect', <module 'inspect' from '/Users/patrickbarker/.pyenv/versions/3.10.1/lib/python3.10/inspect.py'>)
('typing', <module 'typing' from '/Users/patrickbarker/.pyenv/versions/3.10.1/lib/python3.10/typing.py'>)


In [264]:
loca.keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', 'sys', 'os', '__vsc_ipynb_file__', '_i', '_ii', '_iii', '_i1', 'Foo', '_i2', '_2', '_i3', 'Server', '_i4', 'inspect', 'bases', 'base', '_i5', '_5', '_i6', 'init_wrapper', 'req_wrapper', 'ServerMetaClass', '_i7', '_i8', 'List', '_i9', '_i10', 'Dict', '_i11', '_i12', 'Any', '_i13', '_i14', 'wraps', 'FunctionType', '_i15', 'A', '_i16', '_i17', '_i18', 'a', '_i19', '_19', '_i20', '_i21', '_i22', '_i23', '_i24', 'add', '_i25', '_i26', '_26', '_i27', '_i28', '_28', '_i29', '_i30', '_30', '_i31', '_i32', '_i33', 'add1', '_i34', '_i35', '_i36', '_i37', '_37', '_i38', '_i39', '_39', '_i40', '_i41', '_41', '_i42', '_i43', '_43', '_i44', '_i45', '_45', '_i46', '_46', '_i47', '_i48', '_48', '_i49', '_i50', '_50', '_i51', '_51', '_i52', '_i53', '_i54', 'signature', '_i55', '_i56', '_i57', '_i58', '_i59', '_i60', 

In [267]:
inspect.isty(loca["abstractclassmethod"])

type

In [269]:
from modulefinder import ModuleFinder

finder = ModuleFinder()

In [362]:
typ = List
get_origin(typ) == typing.Generic

False

In [ ]:
import importlib
importlib.import_module("")